In [1]:
import numpy as np
import pandas as pd

## Kernels

$$k(\cdot|x_1) = \sigma(y_{i}) + 0.1 \cdot x_1$$

$y_{i} \sim \mathcal{N}(2x_1, 3x_1)$

$$k(\cdot|x_2) = \frac{1}{\sqrt{y_{i}}} \cdot \text{abs}(x_2)$$

$y_{i} \sim \mathcal{N}(x_2^2, 3x_2)$

$$
\alpha, \beta, \lambda , \mu  \\
$$
$$
\alpha > 0  \\
$$
$$
\lambda > 0  \\
$$
$$
\mu \in \mathbb{R}  \\
$$
$$
abs(\beta) < \alpha \\
$$



In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
from numpy.random import default_rng

rng = default_rng(34)
rng2 = default_rng(np.random.randint(1, 3090))
rng3 = default_rng(np.random.randint(1, 3090))
rng4 = default_rng(np.random.randint(1, 3090))

N=4
nu_seed = rng.random((N, 1))
alpha_seed = 13*rng.beta(rng2.random(), rng2.random(), size=(N, 1))
mu_loc = [
    np.random.choice(np.array([-1, 1]))
    * np.random.beta(np.random.random(), np.random.random())
    for _ in range(N)
]
mu_scale = [
    np.random.beta(np.random.random(), np.random.random()) for _ in range(N)
]

mu_seed = rng.normal(loc=mu_loc, scale=mu_scale, size=(N,N))
mu_seed = np.diagonal(mu_seed)[None].T
lambda_seed = rng2.random((N,1))
beta_seed = rng3.random((N,1))

params = np.hstack([mu_seed, lambda_seed, alpha_seed, beta_seed])

In [4]:
import scipy
from scipy.stats import normal_inverse_gamma

signature='(n)->(n)'

def my_sample_from_gamma(x):
    inv_gamma = normal_inverse_gamma(*x)
    x_ = inv_gamma.rvs()[0]
    return x_

sample_from_gamma = np.vectorize(my_sample_from_gamma, signature='(n)->()')

X_train = []
for _ in range(1000):
    params_ = params.copy()
    x = sample_from_gamma(params_)
    X_train.append(x)



In [5]:
X_train = np.array(X_train)

In [6]:
X_train

array([[-1.05244646,  1.58053322,  0.67293   ,  0.30153141],
       [-0.8168701 ,  1.61146652,  0.32621075, -0.37091174],
       [-0.05335237,  1.47827114,  0.39569595,  2.95552267],
       ...,
       [-0.8482688 ,  1.61498051, -0.0091576 ,  0.22030873],
       [-1.34051665,  1.65438493, -0.06116332,  1.47420282],
       [-1.53991293,  1.60448739, -0.00719854,  0.55145059]],
      shape=(1000, 4))

In [ ]:
def kernel_1(x):
    
    

In [ ]:
def generate_kernel_data(
    alpha1, beta1, alpha2, beta2, num_samples=100, samples_per_kernel=4
):
    data_list = []

    for i in range(num_samples):
        #  Sample Seeds (x1 and x2)
        x1 = np.random.normal(loc=alpha1, scale=beta1)
        x2 = np.random.normal(loc=alpha2, scale=beta2)

        # Process x1 Kernel

        # Parameters for the N(mu, sigma) distribution for x1
        mu_1 = 2 * x1
        sigma_1 = abs(3 * x1)

        # Sample 4 values (y) from the distribution
        y_samples_1 = np.random.normal(loc=mu_1, scale=sigma_1, size=samples_per_kernel)

        # Apply the kernel function
        k_values_1 = sigmoid(y_samples_1) + 0.1 * x1

        # Process x2 Kernel

        # Parameters for the N(mu, sigma) distribution for x2
        mu_2 = x2**2
        sigma_2 = abs(3 * x2)

        # Sample 4 values (y) from the distribution
        y_samples_2 = np.random.normal(loc=mu_2, scale=sigma_2, size=samples_per_kernel)

        # This replaces any non-positive y with a small positive number (1e-6)
        y_samples_2[y_samples_2 <= 0] = 1e-6

        # Apply the kernel function: (1 / sqrt(y)) * abs(x2)
        k_values_2 = (1 / np.sqrt(y_samples_2)) * abs(x2)

        row_data = {
            "x1_seed": x1,
            "x2_seed": x2,
        }

        for j in range(samples_per_kernel):
            row_data[f"k1_output_{j + 1}"] = k_values_1[j]
            row_data[f"k2_output_{j + 1}"] = k_values_2[j]

        data_list.append(row_data)

    df = pd.DataFrame(data_list)
    return df

In [15]:
final_data = generate_kernel_data(
    alpha1=5, beta1=10, alpha2=15, beta2=20, num_samples=100, samples_per_kernel=4
)

In [19]:
final_data.head()

,x1_seed,x2_seed,k1_output_1,k2_output_1,k1_output_2,k2_output_2,k1_output_3,k2_output_3,k1_output_4,k2_output_4
0,16.974569,-8.726524,2.697454,1.042234,2.697457,0.861392,1.697462,1.066583,2.697457,0.818430
1,-8.550550,15.657805,0.140813,0.856635,-0.855055,0.959655,-0.855055,0.906347,-0.855055,0.887822
2,7.493067,15.674365,0.760730,0.982754,1.749302,1.141796,0.749920,0.887147,1.749302,0.964876
3,1.156695,17.911992,1.102111,0.847279,1.112339,0.993034,1.091787,0.855107,0.751819,1.015220
4,-9.550308,-5.750766,-0.955024,1.023174,0.044969,0.951915,-0.949947,0.882685,-0.948986,0.688759


In [20]:
# use different distribution